## SETUP

In [4]:
import time
import pandas as pd
# Android ADB
from subprocess import check_output,Popen
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
import sys
import logging
import os
# internal
sys.path.append('./src')
import bot_core
import bot_perception
import port_scan
import bot_handler

import importlib
[importlib.reload(lib) for lib in [bot_core,bot_perception,port_scan,bot_handler]]

logger = logging.getLogger(__name__)
# Check if scrcpy is installed (or install it)
if not bot_handler.check_scrcpy(logger):
    raise Exception("Scrcpy could not be installed")
# Find device (should be emulator-5554 but some bluestacks installations are weird)
device = port_scan.get_device()
if device is None:
    raise Exception("No device found!")
# Start Scrcpy once per restart
if 'started_scrcpy' not in vars():
    started_scrcpy=True
    proc = Popen(['.scrcpy/scrcpy','-s',device], shell=True)
    time.sleep(1) # <-- sleep for 1 second
    proc.terminate() # <-- terminate the process (Scrcpy window can be closed)

# Selected units in deck (order does not matter)
#sel_units= ['crystal.png','chemist.png','monkey.png','dryad.png','hunter.png']
sel_units= ['chemist.png','knight_statue.png','harlequin.png','dryad.png','demon_hunter.png']

bot_handler.select_units(sel_units,show=False)
bot = bot_core.Bot(device)

  0%|          | 0/35639811 [00:00<?, ?it/s]

Scanning 127.0.0.1 Ports 48000 - 65000
Started 5667 threads
Ports Open: [49665, 49668, 49681, 49664, 50689, 50770, 51672, 53194, 54235, 55000, 56970, 57621, 61820, 49666, 49669, 54236, 49667]
Found ADB device! emulator-5554


In [9]:
print(os.listdir("all_units"))

['alchemist.png', 'archer.png', 'banner.png', 'blade_dancer.png', 'bruser.png', 'bruser2.png', 'catapult.png', 'cauldron.png', 'chemist.png', 'clock.png', 'cold_mage.png', 'corsair.png', 'crystal.png', 'cultist.png', 'cultist_off.png', 'demo.png', 'demon_hunter.png', 'demon_hunter2.png', 'dryad.png', 'empty.png', 'engineer.png', 'executioner.png', 'fire_mage.png', 'flinger.png', 'gargoyle.png', 'harlequin.png', 'hex.png', 'hunter.png', 'inquisitor.png', 'ivy.png', 'knight_statue.png', 'lightning.png', 'meteor.png', 'mime.png', 'monkey.png', 'old_icon', 'poisoner.png', 'portal_keeper.png', 'portal_mage.png', 'priest.png', 'pyro.png', 'reaper.png', 'rogue.png', 'scrapper.png', 'sentry.png', 'shaman.png', 'sharpshooter.png', 'sharpstone.png', 'summoner.png', 'thunder.png', 'time_keeper.png', 'trapper.png', 'unit_rank', 'vampire.png', 'wind_arch.png', 'witch_doc.png', 'zealot.png']


## RUN BOT

In [6]:
wait=0
combat = 0
grid_df =None
watch_ad = False
train_ai = False
# Main loop
while(1):
    output = bot.battle_screen(start=False)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Upgrade units
                bot.mana_level([2,3,5],hero_power=True)
                # Spawn units
                bot.click(450,1360)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df,merge_target='demon_hunter.png')
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,unit_series,df_groups])
        # Wait until late stage in combat and if consistency is ok, not stagnate save all units for ML model
        if combat==25 and 5<grid_df['Age'].mean()<50 and train_ai:
            bot_perception.add_grid_to_dataset()
    elif output[1]=='home' and watch_ad:
        for i in range(3):
            bot.watch_ads()
        watch_ad = False
    else:
        combat=0
        output = bot.battle_screen(start=True,pve=True,floor=5) #(only 1,2,4,5,7,8,10 possible)
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR(),
            
            wait=0
        print(output[1],'wait count:',wait)

['2, 8/8 fighting, need more units!',
 'Average age: 9.53',
    grid_pos              unit  u_prob  rank  r_prob   Age
 0    [0, 0]         empty.png   0.000     0     1.0  15.0
 1    [0, 1]         empty.png   0.000     0     1.0   2.0
 2    [0, 2]         empty.png   0.000     0     1.0  15.0
 3    [0, 3]     harlequin.png   0.925     1     1.0   7.0
 4    [0, 4]  demon_hunter.png   0.827     1     1.0   5.0
 5    [1, 0]       chemist.png   0.923     1     1.0  11.0
 6    [1, 1]         dryad.png   0.901     1     1.0   9.0
 7    [1, 2]         dryad.png   0.859     2     1.0   2.0
 8    [1, 3]         dryad.png   0.903     1     1.0  12.0
 9    [1, 4]       chemist.png   0.917     1     1.0   0.0
 10   [2, 0]         empty.png   0.000     0     1.0  15.0
 11   [2, 1]         empty.png   0.000     0     1.0  15.0
 12   [2, 2]         empty.png   0.000     0     1.0  15.0
 13   [2, 3]         dryad.png   0.866     1     1.0   6.0
 14   [2, 4]  demon_hunter.png   0.834     1     1.0  1

KeyboardInterrupt: 

## Quick restart

In [1]:
# Drop co-op game
bot.restart_RR(quick_disconnect=True)

NameError: name 'bot' is not defined

In [ ]:
# Restart game
bot.restart_RR()